# BERT Q/A Model for document search

In [1]:
import os
import re
import glob
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [2]:
def absoluteFilePaths(directory):
    path = []
    files = []
    for dirpath, dirname, filenames in os.walk(directory):
        for f in filenames:
            if not os.path.basename(dirpath).startswith('.'):
                path.append(dirpath)
                files.append(f)
            
    return path, files

In [3]:
# question = "What is attention mechanism"
# true_answer = "The attention mechanism is a part of a neural architecture that enables to dynamically highlight relevant features of the input data, which, in NLP, is typically a sequence of textual elements. It can be applied directly to the raw input or to its higher level representation."

# question = "What is quantum entanglement"
# true_answer = "Quantum Entanglement allows qubits that are separated by incredible distances to interact with each other instantaneously (not limited to the speed of light)."

question = "What are the applications of Face Swapping"
true_answer = "Face swapping has a number of compelling applications in video compositing, transfiguration in portraits, and especially in  identity  protection  as  it  can  replace  faces  in  photographs by ones from a collection of stock images"

In [4]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [5]:
DIRECTORY = os.getcwd()
locations, documents = absoluteFilePaths(os.path.join(DIRECTORY, 'Google', 'research'))
paths = [os.path.join(loc, doc) for loc, doc in zip(locations, documents)]

In [6]:
paths

['/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [7]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [8]:
contents = []
for path in paths:
    if path.endswith('.pdf'):
        contents.append(convert_pdf_to_txt(path))
    else: 
        continue

## BERT Large Cased SQuAD

In [9]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [10]:
paths

['/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [11]:
contents = [re.sub(r'\n', ' ', content) for content in contents]

In [12]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), 50):
        paragraph = content.split(" ")[i:i+50]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [06:27, 96.77s/it] 


In [13]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [14]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6750,6802,6810,8.850172
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,identity protection,1950,1965,1966,8.820515
2,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] Paolo [UNK],17500,17552,17560,8.465274
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,applied to a [UNK] document or to a vocabulary,4000,4011,4019,8.246582
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,[UNK] Creation and [UNK] A Survey,0,14,19,7.6255054


In [15]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+50]))
    print()

/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf
                                      Fig 3: Example to show wave nature and particle nature of light

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
portraits, and especially in identity protection as it can replace faces in photographs by ones from a collection of stock images. However, it is also one of the techniques that cyber attackers employ to penetrate identiﬁcation or authentication systems to gain illegitimate access. The use of deep learning such as

/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf
He is cur- rently an Associate Professor with the Department of Sciences and Methods for Engineering, Univer- sity of Modena and Reggio Emilia, Modena, Italy. His work focuses on machine learning and artiﬁ- cial intelligence, with applications to several areas, including argumentation mining, legal informatics, and medicine.  Paolo Torroni



### Compute F1 scores

In [16]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [17]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6750,6802,6810,8.850172,0.097561
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,identity protection,1950,1965,1966,8.820515,0.117647
2,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] Paolo [UNK],17500,17552,17560,8.465274,0.048780
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,applied to a [UNK] document or to a vocabulary,4000,4011,4019,8.246582,0.000000
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,[UNK] Creation and [UNK] A Survey,0,14,19,7.6255054,0.054054


## BERT Base Cased SQuAD 2

In [18]:
#Model
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')

In [19]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), 50):
        paragraph = content.split(" ")[i:i+50]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [02:08, 32.04s/it]


In [20]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [21]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,require design and build specific,9150,9206,9210,14.402318
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,large scale visual recognition,9300,9352,9355,11.4281025
2,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6750,6802,6810,11.141766
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,metric learning by online soft mining,16300,16333,16338,9.319881
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,videos from appearance and [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UN...,11150,11170,11203,9.22681


In [22]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+50]))
    print()

/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf
answer this question. The contradiction discovered by Bell in EPR paper, is  so  subtle  that  it  appears  only  in  a  very  peculiar  situations  that  had  not  been  investigated. And require design and build specific

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
T., Laine, S., and Lehtinen, J. (2017). Progressive growing of GANs for improved quality, stability, and variation. arXiv preprint arXiv:1710.10196.  [75] Russakovsky, O., Deng, J., Su, H., Krause, J., Satheesh, S., Ma, S., ... and Berg, A. C. (2015). ImageNet large scale visual recognition challenge. International Journal of Computer

/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf
                                      Fig 3: Example to show wave nature and particle nature of light

/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf
TRANSACTIONS ON NEURAL NET

### Compute F1 scores

In [23]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [24]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,require design and build specific,9150,9206,9210,14.402318,0.054054
1,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,large scale visual recognition,9300,9352,9355,11.4281025,0.000000
2,/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf,to show wave nature and particle nature of light,6750,6802,6810,11.141766,0.097561
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,metric learning by online soft mining,16300,16333,16338,9.319881,0.052632
4,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,videos from appearance and [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UN...,11150,11170,11203,9.22681,0.090909


## BERT Large uncased SQuAD

In [25]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [26]:
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), 50):
        paragraph = content.split(" ")[i:i+50]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

4it [06:10, 92.57s/it] 


In [27]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [28]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,fake image detection and face video,2300,2310,2315,11.70063
1,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] [UNK],17500,17551,17558,11.60136
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to detect fake face images,9500,9529,9533,10.294424
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,abusive speech recognition and sentiment,6050,6070,6074,10.155699
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,suggest that the data [UNK] and [UNK] be interpreted in multiple [UNK] [UNK] can be the case whe...,7600,7625,7659,10.107832


In [29]:
for i in range(5):
    print('\033[1m' + df_answers.loc[i, 'path'] + '\033[0m')
    print(" ".join(contents[paths.index(df_answers.loc[i, 'path'])].split(" ")[df_answers.loc[i, 'chunk']: df_answers.loc[i, 'chunk']+50]))
    print()

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
fake image detection and face video detection.  requires a large database of real and fake videos to train clas- siﬁcation models. The number of fake videos is increasingly available, but it is still limited in terms of setting a benchmark for validating various detection methods. To address this issue,

/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf
He is cur- rently an Associate Professor with the Department of Sciences and Methods for Engineering, Univer- sity of Modena and Reggio Emilia, Modena, Italy. His work focuses on machine learning and artiﬁ- cial intelligence, with applications to several areas, including argumentation mining, legal informatics, and medicine.  Paolo Torroni

/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf
(MIPR) (pp. 384-389). IEEE.  [82] Hsu, C. C., Lee, C. Y., and Zhuang, Y. X. (2018, December). Learning to detect fak

### Compute F1 scores

In [30]:
for i in range(5):
    df_answers.loc[i, 'f1'] = compute_f1(true_answer, df_answers.loc[i, 'answer'])

In [31]:
df_answers.head()

,path,answer,chunk,start_loc,end_loc,logit,f1
0,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,fake image detection and face video,2300,2310,2315,11.70063,0.157895
1,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,[UNK] [UNK] legal [UNK] and [UNK] [UNK] [UNK],17500,17551,17558,11.60136,0.050000
2,/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf,to detect fake face images,9500,9529,9533,10.294424,0.108108
3,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,abusive speech recognition and sentiment,6050,6070,6074,10.155699,0.054054
4,/home/jupyter/nlp_document_finder/Google/research/Attention in Natural Language Processing.pdf,suggest that the data [UNK] and [UNK] be interpreted in multiple [UNK] [UNK] can be the case whe...,7600,7625,7659,10.107832,0.125000
